# 개요

NASA에서 공개한 1995년 7월 한 달 간의 웹 액세스 로그를 일별로 나누어 저장하는 코드입니다.

정규표현식을 이용하여 파싱하는 부분은 빅데이터를 지탱하는 기술이라는 책에 나오는 코드를 참고로 하여 작성되었습니다.

# 파싱

일별로 나누기 위해서는 반정형인 로그에 형태를 부여해야 합니다. 정규표현식을 활용하여 액세스 로그를 파싱하여 pandas df에 저장하여 줍니다. <br>
사실 제가 이번에 할 일별로 나눠주는 작업에서 날짜와 인덱스를 제외한 다른 데이터는 필요가 없습니다. 왜냐하면 원본 로그 파일이 시간 순으로 정렬되어 있기 때문에 날짜가 바뀌는 부분의 인덱스만 구할 거니까요. <br>
데이터 자체가 소규모라 이 코드 안에서 모든 전처리와 분석을 끝낼 수도 있습니다만 제 목표는 데이터 파이프라인 구축이기 때문에 여기서는 오직 일자 별로 분할하는 처리만 해줄 겁니다.

In [1]:
import re
import pandas as pd

In [2]:
path = "/source_data/access_log_Jul95"

In [3]:
pattern = re.compile('(\S+) - - \[(.*)\] "(.*)" (\S+) (\S+)$')

In [4]:
def parse_access_log(path):
    for line in open(path, 'rb'):
        line = line.decode(encoding='ascii', errors='ignore')
        for m in pattern.finditer(line):
            yield m.groups()

In [5]:
columns = ["host", 'time', 'request', 'status', 'bytes']

In [6]:
df = pd.DataFrame(parse_access_log(path), columns = columns)
df.head()

,host,time,request,status,bytes
0,199.72.81.55,01/Jul/1995:00:00:01 -0400,GET /history/apollo/ HTTP/1.0,200,6245
1,unicomp6.unicomp.net,01/Jul/1995:00:00:06 -0400,GET /shuttle/countdown/ HTTP/1.0,200,3985
2,199.120.110.21,01/Jul/1995:00:00:09 -0400,GET /shuttle/missions/sts-73/mission-sts-73.ht...,200,4085
3,burger.letters.com,01/Jul/1995:00:00:11 -0400,GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
4,199.120.110.21,01/Jul/1995:00:00:11 -0400,GET /shuttle/missions/sts-73/sts-73-patch-smal...,200,4179


# 시간 형식 처리

In [7]:
df.time = pd.to_datetime(df.time, format='%d/%b/%Y:%X', exact=False)

일자 별로 나누어 저장해주기 위해서는 파이썬에서 처리할 수 있게 datetime 형식으로 바꾸어 주어야 합니다.

In [8]:
df.head()

,host,time,request,status,bytes
0,199.72.81.55,1995-07-01 00:00:01,GET /history/apollo/ HTTP/1.0,200,6245
1,unicomp6.unicomp.net,1995-07-01 00:00:06,GET /shuttle/countdown/ HTTP/1.0,200,3985
2,199.120.110.21,1995-07-01 00:00:09,GET /shuttle/missions/sts-73/mission-sts-73.ht...,200,4085
3,burger.letters.com,1995-07-01 00:00:11,GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
4,199.120.110.21,1995-07-01 00:00:11,GET /shuttle/missions/sts-73/sts-73-patch-smal...,200,4179


이전까지는 문자열로 저장되어 있던 time이 이제는 연산이 가능한 날짜 데이터가 되었습니다.

# 일자 별로 분할

이제 일자 별로 데이터의 개수를 세줄 겁니다. 원본 데이터가 시간 순으로 정렬되어 있기 때문에 일자 별로 세어준 개수대로 잘라주면 원본 데이터를 일자 별로 나눌 수  있습니다.

In [9]:
count_df = df.groupby(df.time.dt.day).count()
count_df

,host,time,request,status,bytes
time,,,,,
1,64714,64714,64714,64714,64714
2,60265,60265,60265,60265,60265
3,89584,89584,89584,89584,89584
4,70452,70452,70452,70452,70452
5,94575,94575,94575,94575,94575
6,100960,100960,100960,100960,100960
7,87233,87233,87233,87233,87233
8,38867,38867,38867,38867,38867
9,35272,35272,35272,35272,35272


일자 별로 개수가 뽑혀져 있습니다. 이제 이 데이터를 리스트로 변환해 줄 겁니다.

In [10]:
counts = list(count_df["host"])

아무 컬럼이나 하나 잡아서 리스트로 변환해줍니다. 데이터프레임을 열로 조회하면 시리즈가 나오는데 시리즈는 리스트로 변환해줄 수 있습니다.

In [11]:
with open(path, "rb") as log:
    lines = log.readlines()

In [12]:
i = 0
for day, cnt in enumerate(counts):
    save_path = "/data/day" + str(day + 1) + ".log"
    with open(save_path, "wb") as day_log:
        for _ in range(cnt):
            day_log.write(lines[i])
            i += 1

일자별로 로그를 나눠 파일에 저장이 완료되었습니다.